In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
import feedback.secret as secret
from feedback.database import open_connection
import openai
import csv

## Donors

In [4]:
db_name = "fr_new" 
username = secret.database_username 
password = secret.database_password 
ip_address = "localhost"
port = secret.database_port

In [5]:
connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [6]:
start_date = "2024-04-26"
end_date = "2024-05-26"

In [7]:
info_file = pd.read_csv('../../results/reports/report_info_{}_{}.csv'.format(start_date,end_date))

In [8]:
info_file

,donor_location_id,donor_location_name,donor_id,donor_name,recipient_location_id,recipient_location_name,recipient_id,recipient_name,volunteer_rating,volunteer_comment
0,6472,Entenmann's Bakery Orange,4539,Bimbo Bakery,10937,Orange VA Clinic,4354,Orange VA Clinic SEE VETERANS ADMIN,4.0,please add Aaron's phone number in notes. thanks!
1,10182,4158 William Penn Highway,6714,Crumbl Cookies,8740,Main,2938,Western Pennsylvania Search and Rescue Develop...,4.0,We were informed that they are making cookings...
2,4618,NaN,3259,Good Shepherd Presbyterian Church,1709,NaN,1344,Precious Life Shelter Inc,1.0,Please take Good Shepherd Church off the pick ...
3,7375,PR - Last Mile Food Rescue,5086,PR - Last Mile Food Rescue,10490,InterParish Ministry - IPM Food Pantry,2060,IPM Food Pantry,4.0,I've never delivered to this pantry before and...
4,10753,South Delta Baptist Church,6343,DELTA FOOD RUNNERS,9737,South Delta Route,3095,DELTA FOOD RUNNERS,4.0,Actually 19 not 20.
...,...,...,...,...,...,...,...,...,...,...
70,123,Settler's Ridge #47,74,Giant Eagle,3284,Main,1663,Sheptytsky Arms/St. George's Close,1.0,Giant eagle says only staurday Sunday and mond...
71,11279,Paris Baguette W 4th Street,7455,Paris Baguette,5423,CAIN,1903,CAIN,3.0,The map coming up in food rescue app and using...
72,12025,210 48th Street,7940,Thoro.ai,1397,Bellefield Dwellings,1098,Bellefield Dwellings,4.0,Deanne not available to accept donation. left...
73,9351,Main Location,6236,Harvie Farms,125,NaN,70,Neighborhood Resilience Project,4.0,This donation was two carloads


In [16]:
client = openai.OpenAI(api_key=secret.openai_api_key)

In [17]:
instruction_json = improve_instructions(client,cursor,info_file)

On 1 out of 75
{'explanation': "The volunteer's feedback suggests adding Aaron's phone number to the instructions, which is considered to be an essential information update for future rescues.", 'donor_info': 'Pick up is in the back of the building, by the loading docks.', 'recipient_info': "Park in smaller side parking lot at entrance. Enter through building side doors. Text/call Aaron Hawkins at [Aaron's Phone Number] upon arrival."}
On 2 out of 75
{'explanation': "The volunteer's feedback indicates a change in the way cookies are made and a decrease in available donations going forward. This information is relevant for future rescues as it impacts the quantity of donations expected from Crumbl Cookies. Therefore, an information update is necessary to reflect this change in the instructions.", 'donor_info': 'Go to front door and let them know you are there for a pick-up. Then drive around back, please!.', 'recipient_info': '', 'info_update': {'donor_info': 'Go to front door and let t

In [18]:
with open('../../results/reports/instruction_update_{}_{}.csv'.format(start_date,end_date),"w",newline="") as f: 
    title = list(instruction_json[0].keys())
    cw = csv.DictWriter(f,title,delimiter=',', quoting=csv.QUOTE_MINIMAL)
    cw.writeheader()
    cw.writerows(instruction_json)